In [13]:
import cv2
import mediapipe as mp
import numpy as np

In [17]:
# Initialize MediaPipe Pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [15]:
# Load the reference video and extract the target keypoints (replace with the appropriate reference video)
reference_video = cv2.VideoCapture('reference video.mp4')
target_keypoints = []  # Store the target keypoints for comparison


In [18]:

# Calculate similarity score between two sets of keypoints
def calculate_similarity_score(keypoints1, keypoints2):
    # Calculate the Euclidean distance between corresponding keypoints
    distances = np.linalg.norm(keypoints1 - keypoints2, axis=1)
    
    # Calculate the average distance as the similarity score
    score = np.mean(distances)
    
    return score

In [20]:
# Initialize the video capture
cap = cv2.VideoCapture(0)
# Parameters for repetition detection
repetition_threshold = 0.5
repetition_count = 0
previous_score = 0
repetition_started = False

while True:
    # Read the frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to RGB and perform pose estimation
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    # Process the pose estimation results to extract body keypoints
    if results.pose_landmarks is not None:
        # Extract keypoints for the body
        keypoints = np.array([[lmk.x, lmk.y, lmk.z] for lmk in results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value:mp_pose.PoseLandmark.RIGHT_HIP.value+1]])
        
        # Store the keypoints as the user's pose
        user_pose = keypoints

    # Extract the target keypoints from the reference video
    ret_ref, frame_ref = reference_video.read()
    if not ret_ref:
        # Restart the reference video if it has ended
        reference_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    # Convert the frame to RGB and perform pose estimation on the reference video frame
    rgb_frame_ref = cv2.cvtColor(frame_ref, cv2.COLOR_BGR2RGB)
    results_ref = pose.process(rgb_frame_ref)

    # Process the pose estimation results to extract body keypoints from the reference video
    if results_ref.pose_landmarks is not None:
        # Extract keypoints for the body from the reference video
        keypoints_ref = np.array([[lmk.x, lmk.y, lmk.z] for lmk in results_ref.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value:mp_pose.PoseLandmark.RIGHT_HIP.value+1]])
        
        # Store the keypoints as the reference pose
        target_keypoints = keypoints_ref

    # Calculate the similarity score between user's pose and reference pose
    if user_pose.shape[0] != 0 and target_keypoints.shape[0] != 0:
        similarity_score = calculate_similarity_score(user_pose, target_keypoints)
        similarity_score = round(similarity_score, 2)
        
        if similarity_score < repetition_threshold and not repetition_started:
            # Start of a repetition
            repetition_started = True
            repetition_count += 1

        if similarity_score >= repetition_threshold and repetition_started:
            # End of a repetition
            repetition_started = False
            
            if repetition_count > 1:
                # Calculate score for the completed repetition
                repetition_score = round((similarity_score - previous_score) / (repetition_count - 1), 2)
                print(f"Repetition {repetition_count - 1} Score: {repetition_score}")

            previous_score = similarity_score

        # Display the similarity score on the frame
        cv2.putText(frame, f"Similarity Score: {similarity_score}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display the frame with keypoints and similarity score
    cv2.imshow("Exercise Grader", frame)

    # Check for user input to exit the application
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release the video capture and close any open windows
cap.release()
cv2.destroyAllWindows()

Repetition 1 Score: -0.05
Repetition 2 Score: -0.01
Repetition 3 Score: -0.0
Repetition 4 Score: 0.0
Repetition 5 Score: -0.0
Repetition 6 Score: 0.0
